# * 머신 러닝의 파이프라인

### 데이터 이해
- 데이터에 대한 기초적인 통계와 시각화, 변수간 관계 확인

### 평가 척도 이해
- 대회의 문제 의도 파악, 패널티 확인

### 교차 검증 기법 선정
- 일정 비율로 훈련/검증 데이터로 분리
- 훈련 데이터로 모델 학습하고 검증 데이터로 평가 → 다수 반복

### 피처 엔지니어링
- 학습에 사용할 데이터 준비: 스케일링, 이상값 제거, 결측값 대체, 범주형 데이터 변환, 변수 선정, 파생 변수 생성 등

### 모델 튜닝
- 교차 검증 점수를 기반으로 모델의 최적 파라미터 파악

### 앙상블
- 다수의 모델을 조합하여 사용


### * Feature engineering
#### (1) 수치로만 구성된 Feature vector 로 만들기
#### (2)  결측값 채우기
- fillna 함수 사용
- 일괄 값으로 채우기: min, mean, max, median 등 사용
- 연관되어 있는 열 정보를 기준으로 값 채우기

#### (3)  Binning(카테고리화하기)
- 연속적인 값으로 의미있는 정보를 찾기 어려울 때, 값을 구간별로 카테고리화하는 것


# 사례: 타이타닉호 생존자 예측

## 1. 문제 정의
- Titanic 호 탑승객의 특징(동승자, 성별, 티켓 클래스 등)에 따른 생존 결과 예측하기
- https://www.kaggle.com/c/2019-1st-ml-month-with-kakr/overview

### * 사용하는 모델
- KNeighborsClassifier(n_neighbors = k)
- DecisionTreeClassifier()
- RandomForestClassifier(n_estimators=k)
- GaussianNB()
- SVC(gamma = 'auto')

In [ ]:
from IPython.display  import Image
Image(url= "https://static1.squarespace.com/static/5006453fe4b09ef2252ba068/t/5090b249e4b047ba54dfd258/1351660113175/TItanic-Survival-Infographic.jpg?format=1500w")

## 2. 데이터 읽어오기

### * 캐글에서 받게되는 파일의 구성
- train.csv - 예측 모델을 만들기 위해 사용하는 학습셋
- test.csv - 예측 모델을 이용하여 예측할 탑승객 정보가 담긴 테스트셋
- submission.csv - 제출시 사용할 수 있는 csv 파일

- https://www.kaggle.com/c/titanic

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
PATH='/content/drive/MyDrive/ICTIS_2023/data/'

In [ ]:
train = pd.read_csv(PATH+'titanic/titanic_train.csv', index_col = 0)
test = pd.read_csv(PATH+'titanic/titanic_test.csv', index_col = 0)

## 3. 데이터 확인

In [ ]:
train.shape

### 데이터 설명
#### * 종속변수 (train 데이터 셋에만 포함되어 있음)
- Survival: 생존유무, target 값. (0 = 사망, 1 = 생존)

#### * 독립변수
- Pclass: 티켓 클래스. (1 = 1st, 2 = 2nd, 3 = 3rd)
- Name: 이름
- Sex: 성별
- Age: 나이(세)
- SibSp (Siblings and spouse): 함께 탑승한 형제자매, 배우자 수 총합
- Parch (Parents and children): 함께 탑승한 부모, 자녀 수 총합
- Ticket: 티켓 넘버
- Fare: 탑승 요금
- Cabin: 객실 넘버
- Embarked: 탑승 항구(C=Cherbourg, Q=Queenstown, S=Southampton)

#### * 주의점: 결측값(NaN) 존재

## 4. 결측값 확인

In [ ]:
train.

In [ ]:
test.

## 5. 이상치 확인

In [ ]:
train.

In [ ]:
test.

## 6.컬럼별 범주형 자료 카테고리 확인

In [ ]:
train.

### * 함수 선언

In [ ]:
def bar_chart(feature):
    global train
    survived = train[    ][feature].value_counts()
    dead = train[  ][feature].value_counts()
    df = pd.DataFrame(   )    
    df.index = ['Survived','Dead']  
    df.plot(kind='bar', stacked=   , figsize=(10,3))
    plt.show()

In [ ]:
def histo(feature):
    global train
    plt.figure(figsize=(10,3) )
    sns.histplot(data=train, x=feature,
             hue= , multiple= , shrink=.8)
    plt.show()

In [ ]:
def bar_histo(feature):
    bar_chart(feature)
    histo(feature)

In [ ]:
train_test_data = [train, test]
def map_col(col, mapping_info):
    for dataset in train_test_data:
        

## 7. 특성별 자료확인 및 결측치 처리

### (1) Survived

In [ ]:
train.Survived

In [ ]:
sns.countplot(data=train, x= )
plt.show()

### (2) Pclass

In [ ]:
train.Pclass

In [ ]:
bar_histo('Pclass')

### (3) Sex

In [ ]:
train.Sex

In [ ]:
bar_histo('Sex')

In [ ]:
g = sns.FacetGrid(train, col=  , row=  )
g.map_dataframe(sns.countplot, x= )
plt.show()

### (4) Age

In [ ]:
train.Age

In [ ]:
plt.figure(figsize=(10,3) )
sns.histplot(data=train, x='Age', binrange=( , ), bins=10, 
             hue= , multiple="dodge", shrink=.8)
plt.show()

### (5) SibSp

In [ ]:
train.SibSp

In [ ]:
bar_histo('SibSp')

### (6) Parch

In [ ]:
train.Parch

In [ ]:
test.Parch

### (7) Fare

In [ ]:
# Fare의 min이 0.0이므로, 0.0의 자료를 찾아봄.
train[  ][   ]

In [ ]:
na_list =
train.iloc[na_list]

In [ ]:
train.iloc[na_list]

In [ ]:
train['Fare'] = 
test['Fare'] = 
train.iloc[na_list]

In [ ]:
#Fare가 0.0인 것을 각 Pclass의 median으로 채워줌
train[].(train.groupby()[].transform(), inplace= )
test
train.iloc[na_list]

In [ ]:
test.Fare

In [ ]:
train.Fare

In [ ]:
train[] = 
train.head()

In [ ]:
bar_histo( 'Fare_Category')

### (8) Cabin

In [ ]:
train.Cabin

In [ ]:
train.Cabin

In [ ]:
train['Cabin_Category'] = train.Cabin.
test
train['Cabin_Category']

In [ ]:
bar_histo('Cabin_Category')

In [ ]:
train.Cabin

In [ ]:
g = sns.FacetGrid(data=, col=, row =)
g.map_dataframe( sns.countplot, x= )
g.add_legend()
plt.show()

In [ ]:
# scaling하여 mapping
cabin_mapping = 
map_col( , )
train.Cabin_Category

In [ ]:
train["Cabin_Category"]
test["Cabin_Category"]

In [ ]:
bar_histo('Cabin_Category')

### (9) Embarked

In [ ]:
train.Embarked

In [ ]:
train.Embarked = train.Embarked
test.Embarked = test.Embarked


In [ ]:
bar_histo('Embarked')

## 8. Feature engineering
- 특성공학 : 컬럼에 연산 등을 통해서 의미있는 새로운 정보를 추출하는 분야

### 8.1 FamilySize 속성 추가

In [ ]:
train['FamilySize'] =
test['FamilySize'] = 

In [ ]:
bar_histo('FamilySize')

In [ ]:
family_mapping = 

### 8.2 범주 데이터를 숫자로 맵핑

### (1) Embarked를 수치로 매핑

In [ ]:
train.Embarked

In [ ]:
e_mapping =

### (2) 성별 데이터를 숫자로 맵핑

In [ ]:
train.Sex

In [ ]:
s_map = 

### 8.3 Name

In [ ]:
train.Name

In [ ]:
train_test_data = [train, test]
# 이름에서 title만 추출하여 Title feature 생성
for dataset in train_test_data:
    str.extract(' ([A-Za-z]+)\.', expand=False)

In [ ]:
(train.Title.value_counts()+ test.Title.value_counts())

In [ ]:
title_mapping = 

train["Title"]
test["Title"]

In [ ]:
bar_histo('Title')

In [ ]:
train.drop('Name', ,  inplace=True)
test.drop('Name',  , inplace=True)

### 8.4 Age 결측값 처리

In [ ]:
train.Age

In [ ]:
train['AgeMedian'] = 
train.head(20)

In [ ]:
# title(Mr, Mrs, Miss, others)에 따른 중앙값
train["Age"]
test["Age"]
train.head(20)

### 8.5 범위에 따라 카테고리화

### (1) Age

In [ ]:
train['Age_Category'] = 
train.head(10)

In [ ]:
test['Age_Category'] = 


### (2) Fare

In [ ]:
train.Fare

In [ ]:
train['Fare_Category'] = 
test['Fare_Category'] = 
train.head()

In [ ]:
bar_histo('Fare_Category')

## 9. 모델 적용

###  (1) 종속 변수와 독립 변수 설정

In [ ]:
features = ['Pclass', 'Sex', 'Age_Category', 'Fare_Category', 
              'Cabin_Category', 'Embarked', 'Title','FamilySize']
x_train =
x_test  =
y_train =

### (2) 교차 검증

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score


k_fold = KFold

def getScore(clf):    
    score = (
                        cv=k_fold, n_jobs=1, scoring='accuracy')
    return score    

In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [ ]:
knn=
dt =
rf =
nb =
svm =

scores = pd.DataFrame(
    ,
    index = ['KNN', 'DT', 'RF', 'NB', 'SVM'] )

In [ ]:
scores['Mean'] =

### (3) 앙상블 &예측

In [ ]:
def getPredict(clf):
    global x_train, y_train, x_test
    

In [ ]:
prediction = pd.DataFrame(
    )

prediction.head(20)

In [ ]:
prediction['Survived'] =

## 10. Submission

In [ ]:
submission = pd.DataFrame({
        "PassengerId":  ,
        "Survived":    
    })
submission.tail()

In [ ]:
submission.to_csv(PATH+'titanic/submission.csv' )